# Exercise 9: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 0. Links 
https://coaxlab.github.io/Data-Explorations/notebooks/limits-of-linear-regression.html

https://coaxlab.github.io/Data-Explorations/notebooks/mixed-effects-models.html

https://coaxlab.github.io/Data-Explorations/notebooks/techniques-for-data-cleansing.html#select-select-columns-to-keep-remove

https://quantifyinghealth.com/linear-regression-with-interaction-in-r/

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [2]:
# WRITE YOUR CODE HERE

library(tidyverse)

setwd("C:/Users/roman/OneDrive/Documents/Python Scripts/DataSciencePsychNeuro-master/Homework datasets/LexDat")

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


### Data Load-in

In [3]:
LexicalData <- read.csv("LexicalData.csv")
head(LexicalData)

LexicalData  <- LexicalData %>%
    rename(Word = D_Word)

Items <- read.csv("Items.csv")
head(Items)

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


### Data Formatting

Remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

In [5]:
LexicalData  <- LexicalData %>%
  mutate(D_RT = as.numeric(gsub(",","",D_RT)))

head(LexicalData)

Items_filtered  <- Items  %>% 
    select(Word, Length, Log_Freq_HAL)


LexicalData  <-  LexicalData %>%
    left_join(Items_filtered, by = "Word") %>%
    drop_na() %>%
    head()

,Sub_ID,Trial,Type,D_RT,Word,Outlier,D_Zscore,Length,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,157,1,1,710,browse,false,-0.437,6,8.856
2,67,1,1,1094,refrigerant,false,0.825,11,4.644
3,120,1,1,587,gaining,false,-0.645,7,8.304
4,21,1,1,984,cheerless,false,0.025,9,2.639
5,236,1,1,577,pattered,false,-0.763,8,1.386
6,236,2,1,715,conjures,false,-0.364,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [23]:
head(LexicalData)

lm_1 = lm(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, data = LexicalData)
summary(lm_1)

,Sub_ID,Trial,Type,D_RT,Word,Outlier,D_Zscore,Length,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,157,1,1,710,browse,false,-0.437,6,8.856
2,67,1,1,1094,refrigerant,false,0.825,11,4.644
3,120,1,1,587,gaining,false,-0.645,7,8.304
4,21,1,1,984,cheerless,false,0.025,9,2.639
5,236,1,1,577,pattered,false,-0.763,8,1.386
6,236,2,1,715,conjures,false,-0.364,8,5.268



Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length, 
    data = LexicalData)

Residuals:
      1       2       3       4       5       6 
  3.266 -36.994 -46.448  71.066 -75.488  84.599 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)
(Intercept)         -2553.14    1155.30  -2.210    0.158
Log_Freq_HAL          400.87     189.63   2.114    0.169
Length                401.69     143.23   2.805    0.107
Log_Freq_HAL:Length   -50.82      25.35  -2.005    0.183

Residual standard error: 103.5 on 2 degrees of freedom
Multiple R-squared:  0.9059,	Adjusted R-squared:  0.7646 
F-statistic: 6.415 on 3 and 2 DF,  p-value: 0.1378


Now, install `lme4` using `install.packages()` and then load the library. 

In [11]:
# WRITE YOUR CODE HERE
install.packages("lme4")
library(lme4)

Installing package into 'C:/Users/roman/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'lme4' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\roman\AppData\Local\Temp\Rtmp8ic3FH\downloaded_packages


Warning message:
"package 'lme4' was built under R version 4.2.3"
Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [24]:
lmer_1 = lmer(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1|Sub_ID), data = LexicalData)
summary(lmer_1)

boundary (singular) fit: see help('isSingular')



Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL:Length + (1 | Sub_ID)
   Data: LexicalData

REML criterion at convergence: 34.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.7290 -0.4258 -0.1629  0.5226  0.8170 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept)     0      0.0   
 Residual             10721    103.5   
Number of obs: 6, groups:  Sub_ID, 5

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         -2553.14    1155.30  -2.210
Log_Freq_HAL          400.87     189.63   2.114
Length                401.69     143.23   2.805
Log_Freq_HAL:Length   -50.82      25.35  -2.005

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.974                
Length      -0.995  0.982         
Lg_Fr_HAL:L  0.957 -0.995   -0.974
optimizer (nloptwrap) convergence code: 0 (OK)
boundary (singular) fit: see help('isSingular')


---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> *Write your response here* 

> HALP THEY'RE THE SAME

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [22]:
# WRITE YOUR CODE HERE

ic = AIC(lm_1, lmer_1)
ic
diff(ic$AIC)

,df,AIC
,<dbl>,<dbl>
lm_1,5,76.11547
lmer_1,6,46.93973


[1] -29.17574

> *Write your response here:*  
> The mixed effects model has a lower AIC, indicating it is superior.

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> *Write your response here*   
> You could control for "Trial", which is the parameter that indicates the order participants answered questions to a group of words. You could use that to control for order effects.

**DUE:** 5pm EST, March 15, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*